In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import math
import matplotlib.dates as mdate
import numpy as np
import datetime as dt
from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns

# final df

In [20]:
df = pd.read_csv('/Users/isdc/Library/CloudStorage/OneDrive-HertieSchool/2024-1/Machine Learning/ML project/machine_learning/all_features_merged.csv')
df.set_index(['Date', 'Country'], inplace=True)

print(df)

                             Solar  Wind Onshore  Wind Offshore  \
Date                Country                                       
2014-12-31 23:00    GB         0.0       4546.00         3165.0   
2015-01-01 00:00    AT         0.0        117.25            NaN   
                    BE         0.0        246.00          420.0   
                    CH         0.0          4.00            NaN   
                    CZ         0.0           NaN            NaN   
...                            ...           ...            ...   
2023-12-31 00:00:00 SE_2       NaN           NaN            NaN   
                    SE_3       NaN           NaN            NaN   
                    SE_4       NaN           NaN            NaN   
                    SI         NaN           NaN            NaN   
                    SK         NaN           NaN            NaN   

                             water_reservoirs_and_hydro_storage  energy_price  \
Date                Country                    

Until 2018, the energy price market had a joint trading zone between Austria, Germany and Luxembourg. After 2018, a separate market appeared in Austria. So we need to merge the columns for them.

In [21]:
df.describe()

,Solar,Wind Onshore,Wind Offshore,water_reservoirs_and_hydro_storage,energy_price,biomass,gas,nuclear
count,1.694963e+06,1.895443e+06,591053.000000,1.196200e+04,3.371340e+06,2.022328e+06,2.227018e+06,820910.000000
mean,3.768433e+02,8.095530e+02,476.024976,3.589091e+06,7.913296e+01,4.019074e+02,1.098890e+03,7085.624705
std,1.153057e+03,1.751954e+03,1042.416130,5.358699e+06,1.196151e+02,1.051868e+03,1.994761e+03,11559.506233
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,-5.000000e+02,0.000000e+00,0.000000e+00,0.000000
25%,0.000000e+00,3.200000e+01,0.000000,3.045920e+05,3.237000e+01,2.775000e+01,2.250000e+01,1595.312500
50%,0.000000e+00,1.710000e+02,91.000000,1.598700e+06,4.794000e+01,7.500000e+01,2.810000e+02,3001.000000
75%,1.932500e+02,7.050000e+02,489.000000,4.036250e+06,7.990000e+01,2.660000e+02,1.204000e+03,6420.000000
max,1.702500e+04,2.015600e+04,9813.000000,3.338800e+07,6.101780e+03,5.368250e+03,2.045400e+04,61490.000000


# Handling missing values

In [22]:
#df.dropna(inplace=True)
#print(df)

We cant drop missing values because we end up with nothing.

In [23]:
# Overall missing values
total_missing = df.isnull().sum().sum()
print(f"Total missing values in the dataset: {total_missing}")

Total missing values in the dataset: 31773487


In [24]:
# Missing values per column
missing_per_column = df.isnull().sum()
print(missing_per_column)

Solar                                 3856100
Wind Onshore                          3655620
Wind Offshore                         4960010
water_reservoirs_and_hydro_storage    5539101
energy_price                          2179723
biomass                               3528735
gas                                   3324045
nuclear                               4730153
dtype: int64


In [28]:
# Heatmap of missing values
missing_data = df.assign(missing_energy_price=df['energy_price'].isnull())

# Pivot this DataFrame to have countries as columns and indices as rows
pivot_table = missing_data.pivot_table(index=df.index, columns='country', values='missing_energy_price', aggfunc='first')

# Plotting the heatmap
plt.figure(figsize=(15, 6))
sns.heatmap(pivot_table, cbar=False, cmap='viridis')
plt.title('Missing Values Heatmap for Energy Price by Country')
plt.show()

KeyError: 'country'

## split data
